# PLSE ETL - Part 2: Scraping PDFs to DB
Note: Using Sample Data Set (count=199)

In [115]:
from PyPDF2 import PdfFileReader

In [116]:
from os import listdir
from os.path import isfile, join

In [117]:
path_to_dockets = '../tmp/sample_dockets'

In [118]:
dockets = [f for f in listdir(path_to_dockets) if isfile(join(path_to_dockets, f))]

In [119]:
keys = ['_c0', 'CountyName', 'DocketNumber', 'FiledDate', 'LastName', 'FirstName', 'MiddleName', 'City', 'State', 'ZipCode', 'OffenseTrackingNumber', 'GenderCode', 'RaceCode', 'BirthDate', 'OriginatingOffenseSequence', 'StatuteType', 'StatuteTitle', 'StatuteSection', 'StatuteSubSection', 'InchoateStatuteTitle', 'InchoateStatuteSection', 'InchoateStatuteSubSection', 'OffenseDisposition', 'OffenseDate', 'OffenseDispositionDate', 'OffenseDescription', 'CaseDisposition', 'CaseDispositionDate', 'OffenseGrade', 'DisposingJudge']

In [120]:
def scrape_pdf(filename):
    path = '{}/{}'.format(path_to_dockets, filename)

    with open(path, 'rb') as f:
        pdf = PdfFileReader(f)
        #info = pdf.getDocumentInfo()
        page_count = pdf.getNumPages()
        
        print(f)

        for page in range(page_count):
            data = pdf.getPage(page).extractText()
            # during development, this body can be extended to run analysis and scraping
            # for "prod," we would need more information about the current backlog size
            print(page)

In [121]:
for docket in dockets:
    # @todo @maybe validate `docket` type and path? or extend the declaration of `dockets` to validate?
    # @todo catch and handle PdfReadError: EOF marker not found, currently breaks loop
    try:
      scrape_pdf(docket)
    except:
      print("Something else went wrong") 

<_io.BufferedReader name='../tmp/sample_dockets/MC-51-CR-0307801-2003.pdf'>
0
1
2
3
4
<_io.BufferedReader name='../tmp/sample_dockets/MC-51-CR-0108981-1996.pdf'>
0
1
2
<_io.BufferedReader name='../tmp/sample_dockets/MC-51-CR-0224291-2002.pdf'>
0
1
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-CR-0100651-2003.pdf'>
0
1
2
3
4
5
6
<_io.BufferedReader name='../tmp/sample_dockets/MC-51-CR-1205031-1994.pdf'>
0
1
2
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-MD-9992548-2002.pdf'>
0
<_io.BufferedReader name='../tmp/sample_dockets/MC-51-SU-0004193-1994.pdf'>
0
1
<_io.BufferedReader name='../tmp/sample_dockets/MC-51-SU-0001804-1985.pdf'>
0
1
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-CR-1003061-1978.pdf'>
0
1
2
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-CR-1200421-1997.pdf'>
0
1
2
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-CR-0805391-1994.pdf'>
0
1
2
3
<_io.BufferedReader name='../tmp/sample_dockets/CP-51-CR-0101801-1975.pdf'>
0
1
2
<_io.Buffere